In [66]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor, CatBoostClassifier
import re
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingRegressor, RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [67]:
VERSION    = 11
VAL_SIZE   = 0.25   # 33%
N_FOLDS    = 5
RANDOM_SEED = 42

# CATBOOST
ITERATIONS = 2000
LR         = 0.05

In [68]:
def fill_owners(row):   
    if row['owners'] == np.NaN:
        if row.mileage < 78500:
            owners = 1
        elif row.mileage < 133000:
            owners = 2
        else:
            owners = 3
        return owners
    else:
        return re.findall(r'\d', row['owners'])[0]

In [69]:
def fill_owners_test(row):
    
    if row['Владельцы'] == np.NaN:
        if row.mileage < 78500:
            owners = 1
        elif row.mileage < 133000:
            owners = 2
        else:
            owners = 3
        return owners
    else:
        return re.findall(r'\d', row['Владельцы'])[0]

In [70]:
def preproc_test(df):
    df = df.drop('brand', axis=1)
    df['name'] = df['name'].apply(lambda x: x.split(' ')[0])
    df.drop(['vehicleConfiguration', 'description', 
             'Комплектация',
              'Руль','Состояние', 'Таможня', 'Владение', 'id'], axis=1, inplace=True)
    df['engineDisplacement'] = df.engineDisplacement.apply(
        lambda x: 0 if x.split(' ')[0] == 'undefined' else int(float(x.split(' ')[0])*10))
    df['mileage'] = df.mileage.apply(lambda x: int(x))
    df['modelDate'] = df.modelDate.apply(lambda x: int(x))
    df['numberOfDoors'] = df.numberOfDoors.apply(lambda x: int(x))
    df['Привод'] = df['Привод'].apply(lambda x: x.lower())
    df['Владельцы'] = df.apply(fill_owners_test, axis=1)
    df['enginePower'] = df.enginePower.apply(lambda x: int(x.split(' ')[0]))
    df['productionDate'] = df['productionDate'].apply(lambda x: int(x))
    return df

def preproc_data(df):
    df = df.drop('brand', axis=1)
    df['name'] = df['name'].apply(lambda x: x.split(' ')[0])
    df.drop(['vehicleConfiguration', 'description', 
             'equipment',
             'name_full',
              'wheel','state', 'customs', 'owningTime', 'Unnamed: 0'], axis=1, inplace=True)
    df['engineDisplacement'] = df.engineDisplacement.apply(
        lambda x: 0 if x.split(' ')[0] == 'undefined' else int(float(x.split(' ')[0])*10))
    df['mileage'] = df.mileage.apply(lambda x: int(re.sub(r'[^\d]', '', x)))
    df['drive'] = df.drive.apply(lambda x: x.lower())
    df['enginePower'] = df.enginePower.apply(lambda x: int(x.split(' ')[0]))
    df['owners'] = df.apply(fill_owners, axis=1)
    return df

def preproc_test_feature_generation(df):
    df = df.drop('brand', axis=1)
    df['name'] = df['name'].apply(lambda x: x.split(' ')[0])
    df.drop(['vehicleConfiguration', 'description', 'Комплектация',
              'Руль','Состояние', 'Таможня', 'Владение', 'id', 
             'color', 'mileage', 'ПТС', 'Владельцы'], axis=1, inplace=True)
    df['engineDisplacement'] = df.engineDisplacement.apply(lambda x: x.split(' ')[0])
    df['modelDate'] = df.modelDate.apply(lambda x: int(x))
    df['numberOfDoors'] = df.numberOfDoors .apply(lambda x: int(x))
    df['Привод'] = df['Привод'].apply(lambda x: x.lower())
    df['enginePower'] = df.enginePower.apply(lambda x: int(x.split(' ')[0]))
    df['productionDate'] = df['productionDate'].apply(lambda x: int(x))
    temp = df.columns.to_list()
    temp[-1] = 'drive'
    df.columns=temp
    return df

In [71]:
cwd = os.getcwd()
df = pd.read_csv(cwd + r'\data\auto_data_x_v2.csv')
dft = pd.read_csv(cwd + r'\data\test.csv')
dft_f = pd.read_csv(cwd + r'\data\test.csv')
from_file = CatBoostClassifier()
from_file.load_model('model_feature_generator_v2')
sample_submission = pd.read_csv(cwd + r'\data\sample_submission.csv')

In [72]:
X_sub = preproc_test(dft)
X = preproc_data(df)
X_for_feature = preproc_test_feature_generation(dft_f)
model = from_file.predict(X_for_feature)
X_sub['model'] = model

In [73]:
#5 modelDate
X['modelDate2'] = 2020 - X.modelDate
X_sub['modelDate2'] = 2020 - X_sub.modelDate
# 9 productionDate
X['productionDate2'] = 2020 - X.productionDate
X_sub['productionDate2'] = 2020 - X_sub.productionDate

In [74]:
y = X.loc[:, 'price']
X.drop('price', axis=1, inplace=True)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [76]:
X_sub.columns = X.columns

In [77]:
cat_features = ['bodyType', 'color', 'fuelType', 'name', 'vehicleTransmission', 'engineDisplacement', 
                'drive', 'pts', 'owners', 'model']

In [78]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations = ITERATIONS,
                              learning_rate = LR,
                              eval_metric='MAPE',
                              random_seed = RANDOM_SEED,)
    model.fit(X_train, y_train,
              cat_features=cat_features,
              eval_set=(X_test, y_test),
              verbose=False,
              use_best_model=True,
              plot=False)
    
    return(model)


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

## Запуск CatBoost

In [79]:
submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(y_train, X_train, X_test, y_test,)
    # score model on test
    test_predict = model.predict(X_test)
    test_score = mape(y_test, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(X_sub)
    model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

1 Fold Test MAPE: 0.121
2 Fold Test MAPE: 0.122
3 Fold Test MAPE: 0.128
4 Fold Test MAPE: 0.124
5 Fold Test MAPE: 0.123

Mean Score: 0.124
Std Score: 0.0024
Max Score: 0.128
Min Score: 0.121


In [80]:
submissions.head(10)

,sub_1,sub_2,sub_3,sub_4,sub_5
0,1.849861e+06,1.860583e+06,1.845055e+06,1.874074e+06,1.775689e+06
1,2.663122e+06,2.756455e+06,2.823938e+06,2.682350e+06,2.476429e+06
2,1.313474e+06,1.409003e+06,1.357749e+06,1.215429e+06,1.269715e+06
3,2.612071e+06,2.841871e+06,2.700843e+06,2.730885e+06,2.712774e+06
4,5.490663e+06,5.693784e+06,5.594006e+06,5.389972e+06,5.212935e+06
5,2.072644e+06,2.108735e+06,2.131787e+06,2.197398e+06,2.132819e+06
6,1.130101e+06,1.191317e+06,1.172653e+06,1.160804e+06,1.208791e+06
7,7.069284e+05,7.479792e+05,7.580403e+05,7.713830e+05,7.389527e+05
8,1.521839e+06,1.608971e+06,1.505706e+06,1.556859e+06,1.534490e+06
9,1.424995e+06,1.497176e+06,1.490885e+06,1.482974e+06,1.436169e+06


In [81]:
submissions['blend_round'] = np.round(((submissions.sum(axis=1))/len(submissions.columns))/1000*0.925)*1000
sample_submission['price'] = submissions['blend_round'].values
sample_submission.to_csv(f'submission_blend_v{VERSION}_round.csv', index=False)
sample_submission.head(10)

,id,price
0,0,1703000.0
1,1,2479000.0
2,2,1215000.0
3,3,2516000.0
4,4,5066000.0
5,5,1969000.0
6,6,1085000.0
7,7,689000.0
8,8,1430000.0
9,9,1356000.0


In [36]:
df.equipment

0       {'Прочее': ['Защита картера'], 'Салон': ['Тони...
1                                                      {}
2       {'Прочее': ['Защита картера'], 'Комфорт': ['Кр...
3                                                      {}
4                                                      {}
                              ...                        
7461                                                   {}
7462    {'Комфорт': ['Круиз-контроль', 'Мультифункцион...
7463    {'Комфорт': ['Круиз-контроль', 'Система «старт...
7464    {'Комфорт': ['Круиз-контроль', 'Мультифункцион...
7465    {'Комфорт': ['Круиз-контроль', 'Мультифункцион...
Name: equipment, Length: 7466, dtype: object

In [63]:
submissions['sub_2']
sample_submission['price'] = np.round(((submissions['sub_2']/1000*0.925)))*1000
sample_submission.to_csv(f'submission_blend_v{VERSION}_best_sub.csv', index=False)

In [62]:
sample_submission['price'] 

0       1727000.0
1       2517000.0
2       1286000.0
3       2534000.0
4       5237000.0
          ...    
3832    1028000.0
3833    2459000.0
3834     500000.0
3835     986000.0
3836    1141000.0
Name: price, Length: 3837, dtype: float64